<a href="https://colab.research.google.com/github/nunzio998/REC-Energy-Management/blob/main/NLP_TASK_Marti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NLP TASK

###IMPORT

In [5]:
import string
from sklearn.model_selection import train_test_split

###LOAD DATASET

In [3]:
%%capture
!pip install datasets

In [4]:
# Load del dataset
from datasets import load_dataset

ds = load_dataset("big_patent")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for big_patent contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/big_patent
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/1207222 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/67068 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/67072 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/75 [00:00<?, ?it/s]

###VISUALIZZAZIONE DATASET

In [6]:
# Visualizzazione dataset
print(ds)

#Esplorazione dataset
print(ds['train'].features)
print(ds['train'].shape)

# print(ds["test"]["description"][1])
# print("\n" + ds["test"]["abstract"][1])

DatasetDict({
    train: Dataset({
        features: ['description', 'abstract'],
        num_rows: 1207222
    })
    validation: Dataset({
        features: ['description', 'abstract'],
        num_rows: 67068
    })
    test: Dataset({
        features: ['description', 'abstract'],
        num_rows: 67072
    })
})
{'description': Value(dtype='string', id=None), 'abstract': Value(dtype='string', id=None)}
(1207222, 2)


###TRAIN/TEST SPLIT DATASET

In [16]:
# Split del datasetTRAIN/TEST SPLIT DATASET
[ ]
# Split del dataset

x_train_text = ds['train']['description']
y_train = ds['train']['abstract']
[7]
16 s


x_train_text = ds['train']['description']
y_train = ds['train']['abstract']

SyntaxError: invalid syntax (<ipython-input-16-96f53bbcec63>, line 8)

In [2]:
x_validation_text = ds['validation']['description']
y_validation = ds['validation']['abstract']

NameError: name 'ds' is not defined

In [ ]:
x_test_text = ds['test']['description']
y_test = ds['test']['abstract']
#print("-----------\n" + x_test_text[1])

###TEXT PREPROCESSING

TEXT CLEANING


*   HTML tags
*   punctuations by negating chars, spaces, and digits
*   consecutive spaces





In [18]:
# Trattamento dei dati tramite regex

import re

def noise_remover(text):
    text = re.sub('<[^>]*>', '', text)  # remove html tags
    text = re.sub('[^\w\s\d]', ' ', text)  # remove punctuations by negating chars, spaces, and digits
    text = re.sub('\s+', ' ', text)  # remove consecutive spaces
    #print(text2)
    return text


In [19]:
%%capture
!pip install nltk

In [20]:
#STOP WORDS

import nltk
from nltk.corpus import stopwords

nltk.download('stopwords') # Download the stopwords module

stopwords = nltk.corpus.stopwords.words(
    "english"
) # Get the list of stop words for English

len(stopwords), stopwords[:10] # print stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(179,
 ['i',
  'me',
  'my',
  'myself',
  'we',
  'our',
  'ours',
  'ourselves',
  'you',
  "you're"])

In [21]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [1]:
# LEMMATIZATION, NOISE REMOVAL, LOWERCASING, TOKENIZATION

from nltk import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# create an object of class LancasterStemmer
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Noise removal
    text = noise_remover(text)
    # Lowercasing
    text = text.lower()
    # Tokenization
    tokens = word_tokenize(text)
    # Stop words removal e lemmatization
    tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.lower() not in stopwords]
    return ' '.join(tokens)

#x_train_text_preprocessed = [preprocess_text(text) for text in x_train_text]
x_validation_text_preprocessed = [preprocess_text(text) for text in x_validation_text]
#x_test_text_preprocessed = [preprocess_text(text) for text in x_test_text]

# Print
for i in range(0):
  print(x_validation_text_preprocessed[i])

NameError: name 'x_validation_text' is not defined

### TEXT REPRESENTATIONS

In [ ]:
# Term Frequency-Inverse Document Frequency (TF-IDF)

from sklearn.feature_extraction.text import TfidfVectorizer
from tabulate import tabulate # Tabulate data to print

print('\nTF-IDF for documents using sklearn')

tfidf = TfidfVectorizer()

x_train_tfidf = tfidf.fit_transform(x_train_text_lemmatized)

# Look at the vocabulary mapping
print("Vocabulary: ", tfidf.vocabulary_)

for i, doc in enumerate(x_train_text_lemmatize):
  print('%s.' % (i+1), doc)
  print(tabulate(round(x_train_tfidf[i], 2).toarray()))

